In [2]:
import mdtraj as md
import pyemma as pm

from pathlib import Path
import os
import pickle
import pandas 
import numpy as np

from openmm import app
import openmm as mm
import openmm.unit as unit
import openmmtools

import openpathsampling as ops
import openpathsampling.engines.openmm as ops_openmm
from openpathsampling.engines.openmm.tools import ops_load_trajectory
from openpathsampling.engines.openmm import trajectory_from_mdtraj

In [25]:
# Paths
title = 'CLN_tis_test'
model_path = Path('./data/CLN_msm/')
md_path = Path('./data/init_paths/CLN_md/')
old_storage_path = Path('./data/CLN_tica-cv-2_tps/CLN_tica-cv-2_tps.nc')
storage_path = Path(f'./data/{title}')
storage_path.mkdir(parents=True, exist_ok=True)

# Parameters
f_scheme = 'ca'
n_mcsteps = 10000

In [11]:
# Read tica_mod, kmeans_mod, and msm

with open(os.path.join(model_path, 'msm_models'), 'rb') as f:
    models = pickle.load(f)
tica_mod = models['tica_mod']
kmeans_mod = models['kmeans_mod']
msm = models['msm']

In [17]:
#%%writefile $storage_path/cvs.py
# Prepare CV functions

def tica_cv(snapshot, tica_mod, f_scheme, element):
    import openpathsampling as ops
    import mdtraj as md
    
    traj = ops.engines.Trajectory([snapshot]).to_mdtraj()
    traj.remove_solvent(inplace=True)
    
    f_traj = md.compute_contacts(traj, scheme=f_scheme)[0]
    tica_traj = tica_mod.transform(f_traj)[0]
    return tica_traj[element]

def circle(snapshot, tica_1, tica_2, center):
    import math
    return math.sqrt((tica_1(snapshot)-center[0])**2 + (tica_2(snapshot)-center[1])**2)

In [19]:
# Prepare CVs
# Define the intersection of volumes using tics

tica_1 = ops.CoordinateFunctionCV("tica_1", tica_cv,
                                  tica_mod=tica_mod,
                                  f_scheme=f_scheme,
                                  element=0)

tica_2 = ops.CoordinateFunctionCV("tica_2", tica_cv,
                                  tica_mod=tica_mod,
                                  f_scheme=f_scheme,
                                  element=1)

circle_folded = ops.CoordinateFunctionCV('circle_folded', circle,
                                         tica_1=tica_1,
                                         tica_2=tica_2,
                                         center=[-0.5,0])

circle_unfolded = ops.CoordinateFunctionCV('circle_unfolded', circle,
                                           tica_1=tica_1,
                                           tica_2=tica_2,
                                           center=[2,0])

# Define metastable state hypervolumes
folded = ops.volume.CVDefinedVolume(collectivevariable=circle_folded, lambda_min=0.0, lambda_max=0.2).named('folded')
unfolded =ops.volume.CVDefinedVolume(collectivevariable=circle_unfolded, lambda_min=0.0, lambda_max=0.3).named('unfolded')

itfc_folded = ops.VolumeInterfaceSet(cv=circle_folded, minvals=0.0, maxvals=[0.2, 0.5, 0.8])
itfc_unfolded = ops.VolumeInterfaceSet(cv=circle_unfolded, minvals=0.0, maxvals=[0.3, 0.75, 1.2])

In [14]:
gro = app.GromacsGroFile(os.path.join(md_path, 'nvt.gro'))
# Gmx topology only stores reference to ff definitions. Need to specifty the directory contains ff files.
top = app.GromacsTopFile(os.path.join(md_path, 'topol.top'), 
                         periodicBoxVectors=gro.getPeriodicBoxVectors(),
                         includeDir=r'/home/rzhu/Loc/gromacs/share/gromacs/top')

# Create system from gmx files
system = top.createSystem(nonbondedMethod=app.PME,
                          nonbondedCutoff=1.0*unit.nanometers,
                          constraints=app.HBonds, 
                          rigidWater=True,
                          ewaldErrorTolerance=0.0005)

# Velocity-Verlet with Velocity Randomisation Integrator that simulates Langvein dynamics
integrator = openmmtools.integrators.VVVRIntegrator(
    # Temperature
    340*unit.kelvin,
    # Collision rate (temperature coupling time constant)
    1.0/unit.picoseconds,
    # Timestep
    2.0*unit.femtoseconds
)
integrator.setConstraintTolerance(0.00001)

In [15]:
# Parallelisation may slow down simulation. It's dependent on the system size (the larger the better), connection speed between gpus (nv>>pcie), gpu power,
# and specific forces. 

openmm_properties = {'DeviceIndex':'2'}
platform = mm.openmm.Platform.getPlatformByName('CUDA')
engine_options = {
    # Allowed returned traj frames
    'n_frames_max': 1000,
    # Time interval between frames = 1 ps
    'n_steps_per_frame': 500
}
ops_topology = ops_openmm.tools.topology_from_pdb(os.path.join(md_path, 'nvt.gro'))
engine = ops_openmm.Engine(
    topology=ops_topology, 
    system=system, 
    integrator=integrator, 
    openmm_properties=openmm_properties,
    options=engine_options
).named('CLN_Openmm_engine')
engine.initialize(platform)

In [23]:
outers = ops.MSOuterTISInterface.from_lambdas({itfc_folded: 1, 
                                               itfc_unfolded: 1.5})

network = ops.MSTISNetwork([(folded, itfc_folded),
                            (unfolded, itfc_unfolded)],
                            ms_outers=outers).named("CLN_tis_network")

In [ ]:
ops.MSTISNetwork()

---
Perform a short equilibration

In [29]:
equilibratoin_scheme = ops.OneWayShootingMoveScheme(network=network, 
                                      selector=ops.UniformSelector(),
                                      engine=engine).named("equilibration_scheme")

In [27]:
old_storage = ops.AnalysisStorage(old_storage_path)
template = old_storage.snapshots[0]
old_sampleset = old_storage.samplesets[0]

NameError: name 'equil_scheme' is not defined

In [30]:
sset = equilibratoin_scheme.initial_conditions_from_trajectories(trajectories=[s.trajectory for s in old_sampleset.samples])

Missing ensembles:
*  [Out folded 0]
*  [Out folded 1]
*  [Out folded 2]
*  [Out unfolded 0]
*  [Out unfolded 1]
*  [Out unfolded 2]
No extra ensembles.
